In [2]:
from os import times_result
import pickle
import pandas as pd



In [3]:
!pip install --upgrade --no-cache-dir gdown

!gdown --folder https://drive.google.com/drive/folders/1ZRJ3Z16RuOYD8tanp27aqOC53fCftG_F

Retrieving folder list
Processing file 17cyEALCv23BSOnHHlhWw0Uiu_dTw4n5_ Bert Classification.ipynb
Processing file 1eKwovBgJiT7QyJBOoZAx-8tpR50BIrhX test.pt
Processing file 1tS306bxHFwyjVtsLIkIuK2fgWLErMqW6 testing.res
Processing file 1SaOuoYAX1UUUNIjDclWSVAZOlyUWFlIi train.pt
Processing file 1Bu-b6h5mbJcbt00dAs7Nwk7JqEhdoSiM training.res
Processing file 11U-O-t8K2_cGbsB3h0WpfYXAYBcXSPZQ valid.pt
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=17cyEALCv23BSOnHHlhWw0Uiu_dTw4n5_
To: /mnt/h/experiments/research_group/classification_2/classification/Bert Classification.ipynb
100%|██████████████████████████████████████| 12.4k/12.4k [00:00<00:00, 33.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1eKwovBgJiT7QyJBOoZAx-8tpR50BIrhX
To: /mnt/h/experiments/research_group/classification_2/classification/test.pt
100%|██████████████████████████████████████| 6.41M/6.41M [00:02<00:00, 2

In [4]:
label2id = {
    'bola': 0,
    'news': 1,
    'bisnis': 2,
    'tekno': 3,
    'otomotif': 4
}

# def perhitungan(nilai):
#   return nilai * 2

def load_data():
  with open("dataset/training.res", "rb") as tdr:
    train_pkl = pickle.load(tdr)
    train = pd.DataFrame({'title': train_pkl[0], 'label': train_pkl[1]})
  
  with open("dataset/testing.res", "rb") as tsdr:
    test_pkl = pickle.load(tsdr)
    test = pd.DataFrame({'title': test_pkl[0], 'label': test_pkl[1]})
  
  train.label = train.label.map(label2id)
  test["label"] = test.label.map(label2id)

  # train["hitung"] = train['label'].apply(lambda x: perhitungan(x))

  return train, test

train, test = load_data()
train

,title,label
0,"Liputan6.com, London - Lee Dixon khawatir Arse...",0
1,"Liputan6.com, Jakarta - Kasus dugaan penganiay...",1
2,"Liputan6.com, Jakarta Menanggapi aksi eks peke...",2
3,"Liputan6.com, Medan - Sebanyak 81 kendaraan 4x...",0
4,"Liputan6.com, Jakarta Indonesia akan melawan T...",0
...,...,...
6122,"Liputan6.com, Jakarta Badan Pusat Statistik (B...",2
6123,"Liputan6.com, Jakarta - PT Waskita Beton Preca...",2
6124,"Liputan6SCTV, Bengkalis - Panglima TNI Marseka...",1
6125,"Liputan6.com, Jakarta - Tahapan wawancara kerj...",2


In [5]:
!pip install Sastrawi

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

## Membersihkan String dari Karakter yang tidak diinginkan

In [6]:
import re

def clean_str(string):
    string = string.lower()
    string = re.sub(r"[^A-Za-z0-9(),!?\'\-`]", " ", string)
    # sebelum = hari ini mendung? | setelah = hari ini mendung ?
    string = re.sub(r"\?", " \? ", string) 
    string = re.sub(r"\!", " \! ", string)
    
    # sebelum = ayam,nasi,sambel | setelah ayam , nasi , sambel
    string = re.sub(r",", " , ", string)
    
    # menghilangkan whitespace / spasi berlebih
    string = re.sub(r"\s{2,}", " ", string)
    
    # menghapus \n / next line
    string = re.sub(r"\n", "", string)
    
    # menghapus \n\t / next line + tab
    string = re.sub(r"n\'t", " n\'t", string)
    
    string = string.strip()
    
    return stemmer.stem(string)
    
    

# Tokenizer

Merubah kata menjadi ID menggunakan pre-trained model BERT

In [7]:
!pip install transformers
!pip install tqdm

In [ ]:
from transformers import BertTokenizer

from tqdm import tqdm
from torch.utils.data import TensorDataset

import torch

tokenizer = BertTokenizer.from_pretrained('indolem/indobert-base-uncased')

max_sentence_length = 100

def arange_data(data, type):
    x_input_ids, y = [], []
    
    for i_baris, kalimat in enumerate(tqdm(data.values.tolist())):
        judul = clean_str(kalimat[0])
        label = kalimat[1]
        
        judul_ids = tokenizer(
            text = judul,
            max_length = max_sentence_length,
            padding = 'max_length',
            truncation = True
        )["input_ids"]
        
        y_label = [0] * len(label2id)
        y_label[int(label)] = 1

        x_input_ids.append(judul_ids)
        y.append(y_label)
        
        # if i_baris > 10:
        #     break
    
    # List to tensor
    x_input_ids = torch.tensor(x_input_ids)
    y = torch.tensor(y)
    
    tensor_dataset = TensorDataset(x_input_ids, y)
    
    # Fine Tunning(Training, Validation), Testing
    # Fine tunning(Training ratio = 0.8, Validation = 0.2)
    
    if type == "train":
        # Pemisahana data untuk training
        train_tensor_dataset, valid_tensor_dataset = torch.utils.data.random_split(tensor_dataset, [
                                                                              round(len(x_input_ids) * 0.8), 
                                                                              len(x_input_ids) - round(len(x_input_ids) * 0.8)])
        torch.save(train_tensor_dataset, "preprocessed/train.pt")
        torch.save(valid_tensor_dataset, "preprocessed/valid.pt")
        
        return train_tensor_dataset, valid_tensor_dataset
    
    else:
        torch.save(tensor_dataset, "preprocessed/test.pt")
        return tensor_dataset
        

train_dataset, validation_dataset = arange_data(data = train, type = "train")
test_dataset = arange_data(data = test, type = "test")

In [ ]:
from torch.utils.data import DataLoader

batch_size = 2

def preprocessor(train_datasets, validation_datasets, test_datasets):
    train_datasets = DataLoader(
        dataset = train_datasets,
        batch_size = batch_size,
        shuffle=True,
        num_workers = 4
    )
    
    validation_datasets = DataLoader(
        dataset = validation_datasets,
        batch_size = batch_size,
        num_workers = 4
    )
    
    test_datasets = DataLoader(
        dataset = test_datasets,
        batch_size = batch_size,
        num_workers = 4 
    )
    
    return train_datasets, validation_datasets, test_datasets

train_datasets, validation_datasets, test_datasets = preprocessor(train_dataset, validation_dataset, test_dataset)
train_datasets

# Mulai Training

In [ ]:
for batch in train_datasets:
    x_input_ids, y = batch
    print(x_input_ids.shape)
    